# Exams Schedule Generator Using Genetic Algorithm

## Project Overview
This project implements a Genetic Algorithm to generate an optimal exam schedule for a university, satisfying both hard and soft constraints.

In [ ]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)


## Data Loading and Preprocessing


In [ ]:
# Load datasets
courses_df = pd.read_csv('Dataset/courses.csv')
student_course_df = pd.read_csv('Dataset/studentCourse.csv')
student_names_df = pd.read_csv('Dataset/studentNames.csv')
teachers_df = pd.read_csv('Dataset/teachers.csv')

# Extract unique values
courses = courses_df['Course Code'].unique().tolist()
students = student_names_df['Names'].unique().tolist()
teachers = teachers_df['Names'].unique().tolist()

print(f"Loaded {len(courses)} courses, {len(students)} students, {len(teachers)} teachers")


In [ ]:
# Analyze student enrollments
student_enrollments = student_course_df.groupby('Student Name')['Course Code'].apply(list).to_dict()
course_enrollments = student_course_df.groupby('Course Code')['Student Name'].apply(list).to_dict()


In [ ]:
# Problem constants
DAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
TIME_SLOTS = ['9:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-13:00', '14:00-15:00', '15:00-16:00', '16:00-17:00']
CLASSROOMS = ['C301', 'C302', 'C303', 'C304', 'C305', 'C306', 'C307', 'C308', 'C309', 'C310', 
              'C401', 'C402', 'C403', 'C404', 'C405', 'C406', 'C407', 'C408', 'C409', 'C410',
              'B227', 'B229', 'B230', 'A301', 'A302', 'A303', 'A310', 'A311', 'A312', 'A314', 'A316']

# Exam duration (in hours)
EXAM_DURATION = 1

# GA Parameters
POPULATION_SIZE = 300
MAX_GENERATIONS = 1000
MUTATION_RATE = 0.15
CROSSOVER_RATE = 0.9
ELITE_SIZE = 30

## Chromosomes Representation

The algorithm creates exam schedules by trying different combinations of:
- Which course
- Which day
- What time
- Which classroom
- Which teacher

Each combination is called a chromosome.


In [ ]:
class ExamSchedule:
    def __init__(self, courses, students, teachers, classrooms, days, time_slots, student_enrollments):
        self.courses = courses
        self.students = students
        self.teachers = teachers
        self.classrooms = classrooms
        self.days = days
        self.time_slots = time_slots
        self.student_enrollments = student_enrollments
        
    def create_random_schedule(self):
        """Create a random exam schedule (chromosome) ensuring all courses are scheduled"""
        schedule = []
        
        courses_to_schedule = self.courses.copy()
        random.shuffle(courses_to_schedule)
        
        for course in courses_to_schedule:
            day = random.choice(self.days)
            time_slot = random.choice(self.time_slots)
            classroom = random.choice(self.classrooms)
            teacher = random.choice(self.teachers)
            
            exam = {
                'course': course,
                'day': day,
                'time_slot': time_slot,
                'classroom': classroom,
                'teacher': teacher
            }
            schedule.append(exam)
        
        return schedule
    
    def create_smart_initial_schedule(self):
        """Create a smarter initial schedule to reduce violations"""
        schedule = []
        
        courses_to_schedule = self.courses.copy()
        random.shuffle(courses_to_schedule)
        
        time_slots_copy = self.time_slots.copy()
        random.shuffle(time_slots_copy)
        
        for i, course in enumerate(courses_to_schedule):
            day = self.days[i % len(self.days)]
            time_slot = time_slots_copy[i % len(time_slots_copy)]
            classroom = self.classrooms[i % len(self.classrooms)]
            teacher = self.teachers[i % len(self.teachers)]
            
            exam = {
                'course': course,
                'day': day,
                'time_slot': time_slot,
                'classroom': classroom,
                'teacher': teacher
            }
            schedule.append(exam)
        
        return schedule
    
    def create_population(self, size):
        """Create initial population with mix of smart and random schedules"""
        population = []
        
        smart_count = max(1, int(size * 0.2))
        
        for i in range(size):
            if i < smart_count:
                schedule = self.create_smart_initial_schedule()
            else:
                schedule = self.create_random_schedule()
            population.append(schedule)
        
        return population

# Initialize the exam schedule system
exam_system = ExamSchedule(courses, students, teachers, CLASSROOMS, DAYS, TIME_SLOTS, student_enrollments)

## Constraint Implementation

### Hard Constraints (Must be satisfied)
1. An exam will be scheduled for each course
2. A student cannot give more than 1 exam at a time
3. Exam will not be held on weekends (already handled by DAYS list)
4. Each exam must be held between 9 am and 5 pm (already handled by TIME_SLOTS)
5. Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time
6. A teacher cannot invigilate two exams in a row
7. A classroom cannot host two exams at the same time

### Soft Constraints (Optimize as much as possible)
1. All students and teachers shall be given a break on Friday from 1-2
2. A student shall not give more than 1 exam consecutively
3. If a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam
4. Two hours of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot

In [ ]:
class ConstraintChecker:
    def __init__(self, courses, students, teachers, student_enrollments):
        self.courses = courses
        self.students = students
        self.teachers = teachers
        self.student_enrollments = student_enrollments
    
    def check_hard_constraints(self, schedule):
        """Check all hard constraints and return violation count"""
        violations = 0
        
        # 1. An exam will be scheduled for each course
        scheduled_courses = {exam['course'] for exam in schedule}
        if len(scheduled_courses) != len(self.courses):
            violations += 1
        
        # 2. A student cannot give more than 1 exam at a time
        time_conflicts = defaultdict(list)
        for exam in schedule:
            key = (exam['day'], exam['time_slot'])
            time_conflicts[key].append(exam)
        
        for (day, time_slot), exams in time_conflicts.items():
            students_in_exams = set()
            for exam in exams:
                course = exam['course']
                if course in course_enrollments:
                    students_in_exams.update(course_enrollments[course])
            
            for student in students_in_exams:
                student_exam_count = 0
                for exam in exams:
                    if exam['course'] in course_enrollments and student in course_enrollments[exam['course']]:
                        student_exam_count += 1
                if student_exam_count > 1:
                    violations += 1
        
        # 3. A teacher cannot invigilate two exams at the same time
        for (day, time_slot), exams in time_conflicts.items():
            teachers_in_slot = [exam['teacher'] for exam in exams]
            if len(teachers_in_slot) != len(set(teachers_in_slot)):
                violations += 1
        
        # 4. A classroom cannot host two exams at the same time
        for (day, time_slot), exams in time_conflicts.items():
            classrooms_in_slot = [exam['classroom'] for exam in exams]
            if len(classrooms_in_slot) != len(set(classrooms_in_slot)):
                violations += 1
        
        # 5. A teacher cannot invigilate two exams in a row
        teacher_schedule = defaultdict(list)
        for exam in schedule:
            teacher_schedule[exam['teacher']].append((exam['day'], exam['time_slot']))
        
        for teacher, slots in teacher_schedule.items():
            day_slots = defaultdict(list)
            for day, time_slot in slots:
                day_slots[day].append(time_slot)
            
            for day, time_slots in day_slots.items():
                time_slots.sort()
                for i in range(len(time_slots) - 1):
                    if self._are_consecutive_slots(time_slots[i], time_slots[i+1]):
                        violations += 1
        
        return violations
    
    def _are_consecutive_slots(self, slot1, slot2):
        """Check if two time slots are consecutive"""
        time_order = ['9:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-13:00', '14:00-15:00', '15:00-16:00', '16:00-17:00']
        try:
            idx1 = time_order.index(slot1)
            idx2 = time_order.index(slot2)
            return abs(idx1 - idx2) == 1
        except ValueError:
            return False
    
    def check_soft_constraints(self, schedule):
        """Check soft constraints and return score (higher is better)"""
        score = 0
        
        # 1. Friday 1-2pm break
        friday_1_2_exams = [exam for exam in schedule if exam['day'] == 'Friday' and exam['time_slot'] == '12:00-13:00']
        if len(friday_1_2_exams) == 0:
            score += 1
        
        # 2. No consecutive student exams
        student_schedules = defaultdict(list)
        for exam in schedule:
            course = exam['course']
            if course in course_enrollments:
                for student in course_enrollments[course]:
                    student_schedules[student].append((exam['day'], exam['time_slot']))
        
        consecutive_violations = 0
        for student, slots in student_schedules.items():
            day_slots = defaultdict(list)
            for day, time_slot in slots:
                day_slots[day].append(time_slot)
            
            for day, time_slots in day_slots.items():
                time_slots.sort()
                for i in range(len(time_slots) - 1):
                    if self._are_consecutive_slots(time_slots[i], time_slots[i+1]):
                        consecutive_violations += 1
        
        if consecutive_violations == 0:
            score += 1
        elif consecutive_violations <= 2:
            score += 0.5
        
        # 3. MG course exam before CS course exam
        mg_cs_violations = 0
        for student, slots in student_schedules.items():
            mg_exams = []
            cs_exams = []
            
            for exam in schedule:
                course = exam['course']
                if course in course_enrollments and student in course_enrollments[course]:
                    if course.startswith('MG'):
                        mg_exams.append((exam['day'], exam['time_slot']))
                    elif course.startswith('CS'):
                        cs_exams.append((exam['day'], exam['time_slot']))
            
            if mg_exams and cs_exams:
                for mg_exam in mg_exams:
                    for cs_exam in cs_exams:
                        if self._is_later_slot(mg_exam, cs_exam):
                            mg_cs_violations += 1
        
        if mg_cs_violations == 0:
            score += 1
        
        # 4. Faculty meeting breaks
        time_usage = defaultdict(int)
        for exam in schedule:
            time_usage[exam['time_slot']] += 1
        
        low_usage_slots = sum(1 for usage in time_usage.values() if usage <= len(self.teachers) // 4)
        if low_usage_slots >= 2:
            score += 1
        
        return score
    
    def _is_later_slot(self, slot1, slot2):
        """Check if slot1 is later than slot2"""
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        time_order = ['9:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-13:00', '14:00-15:00', '15:00-16:00', '16:00-17:00']
        
        day1, time1 = slot1
        day2, time2 = slot2
        
        if day1 != day2:
            return day_order.index(day1) > day_order.index(day2)
        else:
            return time_order.index(time1) > time_order.index(time2)

# Initialize constraint checker
constraint_checker = ConstraintChecker(courses, students, teachers, student_enrollments)

### Fitness Function


In [ ]:
def calculate_fitness(schedule, constraint_checker):
    """Calculate fitness of a schedule"""
    hard_violations = constraint_checker.check_hard_constraints(schedule)
    soft_score = constraint_checker.check_soft_constraints(schedule)
    
    # Count consecutive exam violations manually for better penalty
    consecutive_violations = 0
    time_order = ['9:00-10:00', '10:00-11:00', '11:00-12:00', '12:00-13:00', '14:00-15:00', '15:00-16:00', '16:00-17:00']
    
    day_groups = {}
    for exam in schedule:
        day = exam['day']
        if day not in day_groups:
            day_groups[day] = []
        day_groups[day].append(exam)
    
    for day, day_exams in day_groups.items():
        day_exams.sort(key=lambda x: time_order.index(x['time_slot']))
        
        for i in range(len(day_exams) - 1):
            current_exam = day_exams[i]
            next_exam = day_exams[i + 1]
            
            current_time_idx = time_order.index(current_exam['time_slot'])
            next_time_idx = time_order.index(next_exam['time_slot'])
            
            if next_time_idx == current_time_idx + 1:
                current_course = current_exam['course']
                next_course = next_exam['course']
                
                for student, enrolled_courses in constraint_checker.student_enrollments.items():
                    if current_course in enrolled_courses and next_course in enrolled_courses:
                        consecutive_violations += 1
    
    if hard_violations > 0:
        fitness = 1.0 / (1.0 + hard_violations * 10.0)
    else:
        consecutive_penalty = consecutive_violations * 100.0
        fitness = 1000.0 + soft_score * 100.0 - consecutive_penalty
    
    return fitness, hard_violations, soft_score

## Genetic Algorithm

In [ ]:
class GeneticAlgorithm:
    def __init__(self, exam_system, constraint_checker, population_size, max_generations, 
                 mutation_rate, crossover_rate, elite_size):
        self.exam_system = exam_system
        self.constraint_checker = constraint_checker
        self.population_size = population_size
        self.max_generations = max_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elite_size = elite_size
        
        self.fitness_history = []
        self.best_fitness_history = []
        self.avg_fitness_history = []
    
    def roulette_wheel_selection(self, population, fitness_scores):
        """Select parents using roulette wheel selection"""
        total_fitness = sum(fitness_scores)
        if total_fitness == 0:
            return random.choice(population)
        
        probabilities = [f / total_fitness for f in fitness_scores]
        
        cumulative_probs = []
        cumulative = 0
        for prob in probabilities:
            cumulative += prob
            cumulative_probs.append(cumulative)
        
        r = random.random()
        for i, cum_prob in enumerate(cumulative_probs):
            if r <= cum_prob:
                return population[i]
        
        return population[-1]
    
    def uniform_crossover(self, parent1, parent2):
        """Perform uniform crossover between two parents"""
        if random.random() > self.crossover_rate:
            return parent1, parent2
        
        child1 = []
        child2 = []
        
        for i in range(len(parent1)):
            if random.random() < 0.5:
                child1.append(parent1[i].copy())
                child2.append(parent2[i].copy())
            else:
                child1.append(parent2[i].copy())
                child2.append(parent1[i].copy())
        
        return child1, child2
    
    def mutate(self, individual):
        """Mutate an individual with constraint-aware strategy"""
        if random.random() > self.mutation_rate:
            return individual
        
        mutated = [exam.copy() for exam in individual]
        
        num_mutations = max(1, int(len(mutated) * 0.15))
        
        for _ in range(num_mutations):
            exam_idx = random.randint(0, len(mutated) - 1)
            exam = mutated[exam_idx]
            
            mutation_weights = {
                'time_slot': 0.5,
                'day': 0.3,
                'classroom': 0.15,
                'teacher': 0.05
            }
            
            attribute = random.choices(
                list(mutation_weights.keys()),
                weights=list(mutation_weights.values())
            )[0]
            
            if attribute == 'day':
                exam['day'] = random.choice(self.exam_system.days)
            elif attribute == 'time_slot':
                exam['time_slot'] = random.choice(self.exam_system.time_slots)
            elif attribute == 'classroom':
                exam['classroom'] = random.choice(self.exam_system.classrooms)
            elif attribute == 'teacher':
                exam['teacher'] = random.choice(self.exam_system.teachers)
        
        return mutated
    
    def evolve(self):
        """Run the genetic algorithm"""
        population = self.exam_system.create_population(self.population_size)
        
        print(f"Starting evolution with {self.population_size} individuals...")
        
        for generation in range(self.max_generations):
            fitness_scores = []
            hard_violations_list = []
            soft_scores_list = []
            
            for individual in population:
                fitness, hard_violations, soft_score = calculate_fitness(individual, self.constraint_checker)
                fitness_scores.append(fitness)
                hard_violations_list.append(hard_violations)
                soft_scores_list.append(soft_score)
            
            best_fitness = max(fitness_scores)
            avg_fitness = sum(fitness_scores) / len(fitness_scores)
            
            self.fitness_history.append(fitness_scores)
            self.best_fitness_history.append(best_fitness)
            self.avg_fitness_history.append(avg_fitness)
            
            if generation % 50 == 0 or generation == self.max_generations - 1:
                print(f"Generation {generation:3d}: Best fitness = {best_fitness:.2f}, Avg fitness = {avg_fitness:.2f}")
                print(f"  Hard violations: {min(hard_violations_list)} - {max(hard_violations_list)}")
                print(f"  Soft scores: {min(soft_scores_list)} - {max(soft_scores_list)}")
                print(f"  Valid solutions: {hard_violations_list.count(0)}/{len(hard_violations_list)}")
            
            if hard_violations_list.count(0) > 0 and best_fitness > 1000:
                print(f"\nConverged at generation {generation} with valid solution!")
                break
            
            new_population = []
            
            elite_indices = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i], reverse=True)[:self.elite_size]
            for idx in elite_indices:
                new_population.append(population[idx])
            
            while len(new_population) < self.population_size:
                parent1 = self.roulette_wheel_selection(population, fitness_scores)
                parent2 = self.roulette_wheel_selection(population, fitness_scores)
                
                child1, child2 = self.uniform_crossover(parent1, parent2)
                
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                
                new_population.extend([child1, child2])
            
            population = new_population[:self.population_size]
        
        final_fitness_scores = []
        for individual in population:
            fitness, _, _ = calculate_fitness(individual, self.constraint_checker)
            final_fitness_scores.append(fitness)
        
        best_idx = max(range(len(final_fitness_scores)), key=lambda i: final_fitness_scores[i])
        return population[best_idx], final_fitness_scores[best_idx]

# Initialize GA
ga = GeneticAlgorithm(exam_system, constraint_checker, POPULATION_SIZE, MAX_GENERATIONS, MUTATION_RATE, CROSSOVER_RATE, ELITE_SIZE)

In [ ]:
# Run the genetic algorithm
print("Running Genetic Algorithm for Exam Schedule Generation...")
print("=" * 60)

best_schedule, best_fitness = ga.evolve()

print("\n" + "=" * 60)
print("EVOLUTION COMPLETED")
print("=" * 60)
print(f"Best fitness achieved: {best_fitness:.2f}")

# Evaluate final solution
final_fitness, final_hard_violations, final_soft_score = calculate_fitness(best_schedule, constraint_checker)
print(f"Final hard constraint violations: {final_hard_violations}")
print(f"Final soft constraint score: {final_soft_score}/4")
print(f"All hard constraints satisfied: {final_hard_violations == 0}")

## Final Exam Schedule Output

In [ ]:
# Create detailed schedule output
def format_schedule_output(schedule):
    """Format the schedule for display"""
    schedule_df = pd.DataFrame(schedule)
    schedule_df = schedule_df.sort_values(['day', 'time_slot', 'classroom'])
    
    day_order = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5}
    schedule_df['day_order'] = schedule_df['day'].map(day_order)
    schedule_df = schedule_df.sort_values(['day_order', 'time_slot'])
    
    return schedule_df[['course', 'day', 'time_slot', 'classroom', 'teacher']]

# Save to CSV
final_schedule_df = format_schedule_output(best_schedule)
final_schedule_df.to_csv('final_exam_schedule.csv', index=False)
print(f"Schedule saved to 'final_exam_schedule.csv'")
print(f"Hard constraint violations: {final_hard_violations}")
print(f"Soft constraint score: {final_soft_score}/4")


In [ ]:
# Display the final schedule if you want to see it here
print("FINAL EXAM SCHEDULE")
print("=" * 50)
print(final_schedule_df.to_string(index=False))